In [ ]:
TABLE: customer_order
fields
id
date
store_id

TABLE: customer
id
person_id
job_id
first_time (bool)

https://stackoverflow.com/questions/55649275/sql-retention-cohort-analysis
    

In [ ]:
WITH
cohorts as (
    SELECT person_id, DATE_TRUNC(MIN(CAST(date AS DATE)), month) as first_job_month
    FROM customer_order cd
    JOIN consumer co 
    ON co.job_id = cd.id
    GROUP BY 1
)
,orders as (
    SELECT
     *
    ,round(1.0*(DATE_TRUNC(MIN(CAST(cd.date AS DATE))-c.first_job_month)/30) as months_since_first_order
    FROM cohorts c
    JOIN customer_order cd
    USING (person_id)
)
SELECT
 first_job_month as cohort
,count(distinct person_id) as size
,count(distinct case when months_since_first_order>=1 then person_id end) as m1
,count(distinct case when months_since_first_order>=2 then person_id end) as m2
,count(distinct case when months_since_first_order>=3 then person_id end) as m3
-- hardcode up to the number of months you want and the history you have
FROM orders 
GROUP BY 1
ORDER BY 1